# Install required libraries

In [ ]:
!pip install ultralytics
!pip install roboflow
!pip install -U ipywidgets

# Disable Weights & Biases (wandb) logging

In [ ]:
import os
os.environ['WANDB_MODE'] = 'disabled'

# Import Libraries

In [1]:
from ultralytics import YOLO

# Load Data

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="xTo8OzKt3gqUQENvd7gp")
project = rf.workspace("damage-sverity").project("flowers-krbsa")
version = project.version(2)
dataset = version.download("yolov8")

# Load Model

In [2]:
model = YOLO("yolov8x-seg.pt")

# Training on GPU (Option 01)

In [ ]:
# Train the model
train_results = model.train(
    data="/kaggle/working/Flowers-2/data.yaml",  # path to dataset YAML
    epochs=600,  # number of training epochs
    imgsz=640,  # training image size
    device=0,
    batch=4,
    workers=4,
    seed=101,
    
    # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
)

# Training on GPU (Option 02)

In [ ]:
!yolo segment train model=yolov8x-seg.pt data='/data/data.yaml' workers=8 batch=8 epochs=300 seed=101 device=0 save_period=40 #patience=600 #name=yolov8_car

# Resume training from a checkpoint

In [ ]:
!yolo segment train model=weights/best.pt data='/data/data.yaml' resume=True workers=8 batch=8 epochs=600 seed=101 device=0 save_period=40 #patience=600 #name=yolov8_car

# Export the trained model

In [ ]:
# Export the model to ONNX format
path = model.export(format="onnx")  # return path to exported model

# Evaluate model performance on the validation set

In [ ]:
#Loading Model
model = YOLO("yolov8x-seg.pt")

In [ ]:
metrics = model.val()

# Inference 01

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8x-seg.pt")

image = "/input/53106caea9490a.jpg"
results = model.predict(
    source=image, imgsz=640, vid_stride=1, 
    show=False, save=False, show_labels=True, show_conf=True, 
    show_boxes=False, retina_masks=True, conf=0.10, save_txt=False, 
    save_crop=False
    # line_width=1
)
results[0].show()

# Inference (Option 02)

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8x-seg.pt")

!yolo predict model=yolov8x-seg.pt \
  source='/images/IMG_3190a.jpg' \
  imgsz=640 \
  vid_stride=1 \
  show=True \
  save=True \
  show_labels=True \
  show_conf=True \
  show_boxes=True \
  retina_masks=True \
  conf=0.35 \
  save_txt=False \
  save_crop=True \
  line_width=1


# Display the existing class names in the model

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8x-seg.pt")

model.names

# Change Labels

In [ ]:
import torch

model = torch.load(r"c:\Users\Sakib Ahmed\Desktop\model_car_parts.pt", map_location="cpu")

model["model"].names

In [4]:
# Dictionary mapping old class names to new class names
car_parts_class_mapping = {
    0: 'Alloy Rim Back Left Side',
    1: 'Antenna',
    2: 'Back Bumper',
    3: 'Alloy Rim Front Right Side',
    4: 'Antenna',
    5: 'Brake Light',
}

# Update the model with new class names
model["model"].names = car_parts_class_mapping

# Check the updated class names

In [ ]:
model["model"].names

# Save the model with updated class names

In [6]:
torch.save(model, "model_car_parts.pt")